## BFS Parents Algorithm (Julia - Graph Blas Library)

In [3]:
using SuiteSparseGraphBLAS
using SparseArrays
using LinearAlgebra

In [501]:
function insert(p, result)
    for i=1:length(p)
        if(p[i]!=nothing && result[i]==nothing)
            result[i] = p[i]
        end
    end
    
    return result
    
end
        

insert (generic function with 2 methods)

In [536]:
function updateVisited(f, visited)
    for i=1:length(f)
        if(f[i]!=nothing)
            visited[i] = true
        end
    end
    
    return visited
    
end
        

updateVisited (generic function with 1 method)

In [1]:
#A is the input matrix, s is the source node, and n is the number of nodes
function bfs_parents(A, s, n)
     
    index = GBVector{Int64}(n)
        for i = 1:n
            index[i] = i
        end
    
    #wavefront vector -> w[s]=source node insrted in wavefront
    f = GBVector{Int64}(n)
    f[s] = 1

    #parent vector -> p[s]=1 because the source is already visited
    p = GBVector{Int64}(n)
    
    #result vector -> result[s]=0 because the parent of the source is considered node 0
    result = GBVector{Int64}(n)
    result[s] = 0
    
    #visited vector -> v[s]=1 because the source is already visited
    visited = GBVector{Bool}(n)
    visited[s] = true
    
    print("STARTING ...\n\n")
    for i = 1:n-1
            f = mul(f, A, Semirings.MIN_FIRST, mask=p, desc=Descriptors.SC)
            p = f[:, mask=f, desc=Descriptors.S]
            f = index[:, mask=f, desc=Descriptors.S]
            result = insert(p, result)
            visited = updateVisited(f, visited)
            unvisited = filter(x -> x!=true, visited)
            if(isempty(unvisited))
                break
            end
            print("\nITERATION FINISHED\n\n")
            print(result)
            
    end
    
    print("COMPLETED\n\n")
    print("RESULT\n\n")
    print(result')
    
end

bfs_parents (generic function with 1 method)

## Test

In [551]:
#0, 1, 1, 1, 0, 0, 0, 
#0, 0, 0, 0, 0, 0, 0, 
#0, 0, 0, 0, 0, 0, 0, 
#0, 0, 0, 0, 0, 1, 0, 
#0, 0, 0, 0, 0, 0, 0, 
#0, 0, 0, 0, 0, 0, 1, 
#0, 0, 0, 0, 1, 0, 0, 

#[0, 0, 0, 1, 0, 0, 0] [1, 0, 0, 0, 0, 0, 0] [0, 0, 0, 1, 0, 1, 1] [1, 0, 0, 0, 0, 0, 1] [0, 1, 0, 0, 0, 0, 1] [0, 0, 1, 0, 1, 0, 0] [0, 1, 0, 0, 0, 0, 0]


matrix =  GBMatrix(sparse([[0, 0, 0, 1, 0, 0, 0] [1, 0, 0, 0, 0, 0, 0] [0, 0, 0, 1, 0, 1, 1] [1, 0, 0, 0, 0, 0, 1] [0, 1, 0, 0, 0, 0, 1] [0, 0, 1, 0, 1, 0, 0] [0, 1, 0, 0, 0, 0, 0]]))
@time bfs_parents(matrix, 1, 7)


STARTING ...

7x1 GraphBLAS int64_t vector, bitmap by col
  2 entries, memory: 328 bytes

    (2,1)   2
    (4,1)   4

ITERATION FINISHED

7x1 GraphBLAS int64_t vector, bitmap by col
  3 entries, memory: 328 bytes

    (1,1)   0
    (2,1)   1
    (4,1)   1
7x1 GraphBLAS int64_t vector, bitmap by col
  4 entries, memory: 328 bytes

    (1,1)   1
    (3,1)   3
    (5,1)   5
    (7,1)   7

ITERATION FINISHED

7x1 GraphBLAS int64_t vector, bitmap by col
  6 entries, memory: 328 bytes

    (1,1)   0
    (2,1)   1
    (3,1)   4
    (4,1)   1
    (5,1)   2
    (7,1)   2
7x1 GraphBLAS int64_t vector, bitmap by col
  3 entries, memory: 328 bytes

    (2,1)   2
    (4,1)   4
    (6,1)   6
COMPLETED

RESULT

[0 1 4 1 2 3 2]  0.070613 seconds (129.99 k allocations: 7.562 MiB, 96.72% compilation time)
